In [ ]:

from matplotlib.pyplot import *
from numpy import *
set_printoptions(legacy = '1.25')

from numpy.random import default_rng as rng
samples = rng().random
N, d = 20, 2

# Nxd array
dataset = samples((N,d))

mu = mean(dataset, axis = 0)
p = samples((2,))

for x in dataset:
	points = array([mu, x])
	plot(*points.T, c = 'green', lw = .5)
	points = array([p, x])
	plot(*points.T, c = 'red', lw = .5)

scatter(*mu, s = 10)
scatter(*dataset.T, s = 10)

grid()
show()


In [ ]:

def tensor(u,v): 
	return array([ [ a*b for b in v ] for a in u ])
	
N, d = 20, 2
# N x d array
dataset = samples((N,d))
mu = mean(dataset, axis = 0)
	
# center dataset
A = dataset - mu
	
Qtensor = mean([ tensor(v,v) for v in A ], axis = 0)
Qtensor


In [ ]:

Qcov = cov(dataset.T, bias = True)
Qcov


In [ ]:

corrcoef(dataset.T)


In [ ]:

trace(Qcov)	


In [ ]:

u = array([1,1])/sqrt(2)

# project along unit vector u
q = dot(u, dot(Qcov, u))
q


In [ ]:

from scipy.linalg import inv
	
def ellipse(Q, mu, padding = .5, levels = [1], render = 'var'):
	scatter(*mu, c = 'red', s = 5)
	a, b, c = Q[0,0], Q[0,1], Q[1,1]
	d, e = mu
	delta = .01
	# features (s,t)
	s = arange(d-padding, d+padding, delta)
	t = arange(e-padding, e+padding, delta)
	s, t = meshgrid(s, t)
	if  render == 'var' or render == 'both': 
		# matrix_text(Q, mu, padding, 'var')
		eq = a*(s-d)**2 + 2*b*(s-d)*(t-e) + c*(t-e)**2
		contour(s, t, eq, levels = levels, colors = 'blue', linewidths = .5)
	if render == 'inv' or render == 'both':
		draw_major_minor_axes(Q, mu)
		Q = inv(Q)
		# matrix_text(Q, mu, padding, 'inv')
		A, B, C = Q[0,0], Q[0,1], Q[1,1]
		eq = A*(s-d)**2 + 2*B*(s-d)*(t-e) + C*(t-e)**2
		contour(s, t, eq, levels = levels, colors = 'red', linewidths = .5)


In [ ]:

rcParams['text.usetex'] = True
rcParams['text.latex.preamble'] = r'\usepackage{amsmath}'

# to display the matrix Q, 
# uncomment matrix_text in ellipse above

def matrix_text(Q, mu, padding, render = 'inv'):
	Q = round(Q, 2)
	a, b, c = Q[0,0], Q[0,1], Q[1,1]
	d, e = mu
	vloc = e + 3*padding/4
	if render == 'var': 
		hloc = d - padding/2; tex = '$Q='
		color = 'blue'
	else: hloc = d; tex = '$Q^{-1}='; color = 'red'
	# r'...' means raw string
	tex += r'\begin{pmatrix}'
	tex += f' {a} & {b} ' + r'\\' + f' {b} & {c} ' 
	tex += r'\end{pmatrix}$'
	return text(hloc, vloc, tex, size = 15, c = color)


In [ ]:

def draw_major_minor_axes(Q,mu):
	a, b, c = Q[0,0], Q[0,1], Q[1,1]
	d, e = mu
	label = { 1:'major', -1:'minor' }
	for pm in [1,-1]:
		lamda = (a+c)/2 + pm * sqrt(b**2 + (a-c)**2/4)
		sigma = sqrt(lamda)
		lenv = sqrt(b**2 + (a-lamda)**2)
		lenw = sqrt(b**2 + (c-lamda)**2)
		if lenv: deltaX, deltaY = b/lenv, (a-lamda)/lenv
		elif lenw: deltaX, deltaY = (lamda-c)/lenw, b/lenw
		elif pm == 1:  deltaX, deltaY = 1, 0
		else:  deltaX, deltaY = 0, 1
		axesX = [d+sigma*deltaX, d-sigma*deltaX]
		axesY = [e-sigma*deltaY, e+sigma*deltaY]
		plot(axesX, axesY, lw = .5, label = label[pm])
	legend()


In [ ]:
 
mu = array([0,0])
Q = array([[9,0],[0,4]])
ellipse(Q, mu, padding = 4, render = 'both')

grid()
show()

Q = array([[9,2],[2,4]])
ellipse(Q, mu, padding = 4, render = 'both')

grid()
show()


In [ ]:

N, d = 50, 2
# Nxd array
dataset = samples((N,d))
Q = cov(dataset.T, bias = True)
mu = mean(dataset, axis = 0)
print(f'mu = {mu}') 
print(f'Q = {Q}')

scatter(*dataset.T, s = 5)
ellipse(Q, mu, render = 'var', padding = .5, levels = [.005,.01,.02])
grid()
show()

scatter(*dataset.T, s = 5)
ellipse(Q, mu, render = 'inv', padding = .5, levels = [.5,1,2])
grid()
show()


In [ ]:

d = 10
# 100 x 2 array
dataset = array([ array([i+j,j]) for i in range(d) for j in range(d) ])
